In [1]:
import os

In [2]:
import geopandas as gpd

In [3]:
import pandas as pd

In [4]:
os.chdir('D:\Kaggle\Geospital course')

In [5]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [6]:
crimes = pd.read_csv('crimes-in-boston/crimes-in-boston/crime.csv', encoding='latin-1')

crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin(['Larcny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 
                                               'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary', 
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment', 
    'Manslaughter'])]

crimes = crimes[crimes.YEAR>=2018]

crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
6,I182070933,724,Auto Theft,AUTO THEFT,B2,330,NaN,2018-09-03 21:25:00,2018,9,Monday,21,Part One,NORMANDY ST,42.306072,-71.082733,"(42.30607218, -71.08273260)"
8,I182070931,301,Robbery,ROBBERY - STREET,C6,177,NaN,2018-09-03 20:48:00,2018,9,Monday,20,Part One,MASSACHUSETTS AVE,42.331521,-71.070853,"(42.33152148, -71.07085307)"
19,I182070915,614,Larceny From Motor Vehicle,LARCENY THEFT FROM MV - NON-ACCESSORY,B2,181,NaN,2018-09-02 18:00:00,2018,9,Sunday,18,Part One,SHIRLEY ST,42.325695,-71.068168,"(42.32569490, -71.06816778)"
24,I182070908,522,Residential Burglary,BURGLARY - RESIDENTIAL - NO FORCE,B2,911,NaN,2018-09-03 18:38:00,2018,9,Monday,18,Part One,ANNUNCIATION RD,42.335062,-71.093168,"(42.33506218, -71.09316781)"
27,I182070904,802,Simple Assault,ASSAULT SIMPLE - BATTERY,C11,242,NaN,2018-09-03 18:34:00,2018,9,Monday,18,Part Two,ANNAPOLIS ST,42.317319,-71.061509,"(42.31731905, -71.06150882)"


In [7]:
daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == 'Robbery') & \
                            (crimes.HOUR.isin(range(9,18))))]

In [8]:
# Location where crime occured
m_2 = folium.Map(location=[42.32, -71.0598], tiles='cartodbpositron', zoom_start=13)

for idx, row in daytime_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(m_2)
    
m_2

In [9]:
import math

In [10]:
# Cluster Map
m_3 = folium.Map(location=[42.32, -71.0589], tiles='cartodbpositron', zoom_start=13)

mc = MarkerCluster()
for idx, row in daytime_robberies.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']]))

m_3.add_child(mc)

m_3

In [11]:
# crime commited at day/night classifier map
m_4 = folium.Map(location=[42.32, -71.0589], tiles='cartodbpositron', zoom_start=13)

def color_producer(val):
    if val <= 12:
        return 'forestgreen'
    else:
        return 'darkred'
    
for i in range(0, len(daytime_robberies)):
    Circle(
    location=[daytime_robberies.iloc[i]['Lat'], daytime_robberies.iloc[i]['Long']],
    radius=20,
    color=color_producer(daytime_robberies.iloc[i]['HOUR'])).add_to(m_4)

m_4

# green = day
# red = night

In [12]:
#Heatmap
m_5 = folium.Map(location=[42.32, -71.0589], tiles='cartodbpositron', zoom_start=12)

HeatMap(data=crimes[['Lat','Long']], radius=10).add_to(m_5)

m_5